# Lesson 4-5: Multi-Agent Patterns

This notebook covers two ways to orchestrate multiple agents:

1. **Handoffs** - Decentralized delegation (agent transfers control to another)
2. **Agents-as-Tools** - Centralized orchestration (manager calls agents as tools)

## The Key Question

When should you use **handoffs** vs **agents-as-tools**?

| Pattern | Control Flow | Best For |
|---------|--------------|----------|
| Handoffs | Transfer control completely | Specialist completes the entire task |
| Agents-as-Tools | Manager stays in control | Combining results from multiple specialists |

## Setup

In [1]:
import nest_asyncio
nest_asyncio.apply()

import os
import getpass

if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key: ")

## Part 1: Handoffs (Decentralized Delegation)

A **handoff** transfers the entire conversation to another agent. The new agent:
- Gets the full conversation history
- Takes over completely
- Produces the final response

Think of it like transferring a phone call to a specialist.

In [2]:
from agents import Agent, Runner

# Create specialist agents
math_tutor = Agent(
    name="MathTutor",
    instructions="""You are a math tutor. Help students understand math concepts.
    - Explain step by step
    - Use examples
    - Be encouraging""",
    model="gpt-4.1",
    handoff_description="Hand off for math questions (algebra, calculus, geometry, etc.)"
)

history_tutor = Agent(
    name="HistoryTutor",
    instructions="""You are a history tutor. Help students understand historical events.
    - Provide context and background
    - Discuss causes and effects
    - Make connections to modern times""",
    model="gpt-4.1",
    handoff_description="Hand off for history questions (events, periods, historical figures)"
)

# Create triage agent that routes to specialists
triage_agent = Agent(
    name="TutorTriage",
    instructions="""You are the front desk of a tutoring center.
    Determine what subject the student needs help with and hand off to the appropriate tutor.
    If the question isn't about math or history, politely explain we only offer those subjects.""",
    model="gpt-4.1",
    handoffs=[math_tutor, history_tutor]  # <-- Handoffs go here
)

In [3]:
# Test with a math question
result = Runner.run_sync(triage_agent, "Can you help me solve 2x + 5 = 13?")
print(result.final_output)

Absolutely! Let's solve the equation step by step:

Equation: 2x + 5 = 13

Step 1: Subtract 5 from both sides to get rid of the +5.
2x + 5 - 5 = 13 - 5

That simplifies to:
2x = 8

Step 2: Divide both sides by 2 to solve for x.
2x ÷ 2 = 8 ÷ 2

Which gives:
x = 4

So the solution is x = 4!

Great work! If you want to check your answer, just plug 4 back into the original equation:
2(4) + 5 = 8 + 5 = 13

And that's correct! Let me know if you want to try another one.


In [4]:
# Test with a history question
result = Runner.run_sync(triage_agent, "What caused World War I?")
print(result.final_output)

World War I, also known as the Great War, started in 1914 and lasted until 1918. It involved the major powers of Europe—and eventually countries all over the world. Here’s how and why it started:

**Context and Background:**
By the early 1900s, Europe was divided into two main alliances:
- The Triple Entente (France, Russia, Britain)
- The Triple Alliance (Germany, Austro-Hungarian Empire, Italy)

Countries were also competing for colonies, growing militaries, and fostering intense nationalism (pride in their nation or ethnic group).

**Key Causes:**

1. **Nationalism:** Many groups sought independence (especially in the Balkans), and nations often saw themselves as superior—fueling tensions.
2. **Imperialism:** European powers competed fiercely for colonies, particularly in Africa and Asia, creating international rivalries.
3. **Militarism:** Nations built up huge militaries and developed detailed war plans, making them ready and eager for war.
4. **Alliances:** Secret treaties and de

In [5]:
# Test with an unsupported subject
result = Runner.run_sync(triage_agent, "Can you help me with my chemistry homework?")
print(result.final_output)

I'm sorry, but we currently only offer tutoring in math and history. If you need help with either of those subjects, I'd be happy to connect you with one of our tutors!


## Visualizing the Agent Graph

The SDK includes a visualization tool to see agent relationships.

In [6]:
# Note: Requires graphviz to be installed
# pip install openai-agents[viz]
try:
    from agents.extensions.visualization import draw_graph
    draw_graph(triage_agent)
except ImportError:
    print("Install visualization support: pip install openai-agents[viz]")

## How Handoffs Work Under the Hood

When you define `handoffs=[agent1, agent2]`, the SDK:

1. Creates a tool for each agent: `transfer_to_math_tutor`, `transfer_to_history_tutor`
2. Uses `handoff_description` as the tool description
3. When the LLM calls the transfer tool, the new agent takes over
4. The new agent receives the full conversation history

## Part 2: Agents-as-Tools (Centralized Orchestration)

With `agent.as_tool()`, you call another agent like a function:
- The manager agent stays in control
- Gets results back from sub-agents
- Can call multiple agents and synthesize results

Think of it like a manager delegating tasks to team members.

In [7]:
# Create specialist agents
summarizer = Agent(
    name="Summarizer",
    instructions="""You summarize text concisely.
    - Extract key points
    - Keep it brief (2-3 sentences)
    - Maintain accuracy""",
    model="gpt-4.1"
)

fact_checker = Agent(
    name="FactChecker",
    instructions="""You verify factual claims.
    - Identify factual statements
    - Note any potential issues or inaccuracies
    - Be skeptical but fair""",
    model="gpt-4.1"
)

# Create a manager that uses agents as tools
research_manager = Agent(
    name="ResearchManager",
    instructions="""You are a research manager. When given content to analyze:
    1. Use the summarizer to get a concise summary
    2. Use the fact_checker to verify claims
    3. Combine both results into a final report""",
    model="gpt-4.1",
    tools=[
        summarizer.as_tool(
            tool_name="summarize",
            tool_description="Summarize the given text concisely"
        ),
        fact_checker.as_tool(
            tool_name="check_facts",
            tool_description="Verify factual claims in the text"
        )
    ]
)

In [8]:
content = """
The Great Wall of China is the longest wall in the world, stretching over 13,000 miles.
It was built entirely during the Ming Dynasty and took only 50 years to complete.
The wall is visible from space with the naked eye. It was primarily built to keep out
Mongolian invaders and features watchtowers every 100 meters.
"""

result = Runner.run_sync(research_manager, f"Analyze this content:\n{content}")
print(result.final_output)

Final Report:

Summary:
The Great Wall of China extends over 13,000 miles and was intended primarily to protect against Mongolian invaders, incorporating numerous watchtowers. However, several common beliefs about the wall are inaccurate: it was not built entirely during the Ming Dynasty, did not take only 50 years to complete, and is not visible from space with the naked eye. Construction spanned multiple dynasties over centuries.

Fact-Check Findings:
1. Claim that the Great Wall is the longest wall in the world at over 13,000 miles is accurate.
2. Assertion that it was built entirely during the Ming Dynasty in just 50 years is false; construction began centuries earlier and the Ming section alone took over 200 years.
3. The notion that the wall is visible from space with the naked eye is a myth.
4. Its primary purpose was to repel Mongolian (and other steppe) invaders during the Ming, but earlier walls had other purposes.
5. Claim of watchtowers every 100 meters is exaggerated; thei

Notice how the manager:
1. Called the summarizer
2. Called the fact checker
3. Combined the results into a coherent report

The manager **stayed in control** throughout - unlike handoffs where control transfers completely.

## Decision Framework: Handoffs vs Agents-as-Tools

### Use Handoffs When:
- The specialist should **complete the entire task**
- User should **interact directly** with the specialist
- No need to **return to the original agent**
- Example: Customer support triage → specialist handles the whole issue

### Use Agents-as-Tools When:
- Manager needs to **coordinate multiple specialists**
- Results need to be **combined or synthesized**
- Manager should **maintain conversation control**
- Example: Research manager combining summaries and fact-checks

In [9]:
# Let's see another agents-as-tools example: parallel analysis

sentiment_analyst = Agent(
    name="SentimentAnalyst",
    instructions="Analyze the sentiment of text. Output: positive, negative, or neutral with brief reasoning.",
    model="gpt-4.1"
)

keyword_extractor = Agent(
    name="KeywordExtractor",
    instructions="Extract the 3-5 most important keywords from the text.",
    model="gpt-4.1"
)

content_analyzer = Agent(
    name="ContentAnalyzer",
    instructions="""Analyze content by:
    1. Getting sentiment analysis
    2. Extracting keywords
    3. Providing a combined analysis report""",
    model="gpt-4.1",
    tools=[
        sentiment_analyst.as_tool(
            tool_name="analyze_sentiment",
            tool_description="Analyze the sentiment of text"
        ),
        keyword_extractor.as_tool(
            tool_name="extract_keywords",
            tool_description="Extract key terms from text"
        )
    ]
)

In [10]:
review = "This product exceeded my expectations! The build quality is fantastic and customer service was incredibly helpful when I had questions."

result = Runner.run_sync(content_analyzer, f"Analyze this review:\n{review}")
print(result.final_output)

Combined Analysis Report:

1. Sentiment Analysis:
   - The review is overwhelmingly positive. The user expresses strong satisfaction with both the product quality and customer service, indicating a highly favorable experience.

2. Extracted Keywords:
   - Build quality
   - Customer service
   - Exceeded expectations

3. Summary:
   - The reviewer highlights exceptional build quality and helpful customer service as standout features. Their overall sentiment is very positive, suggesting they are pleased with their purchase and support experience.


## Combining Both Patterns

You can use handoffs AND agents-as-tools in the same system.

Example: A customer service system where:
- Triage agent **hands off** to specialists (handoff pattern)
- Each specialist **uses tools** for specific subtasks (agents-as-tools)

In [11]:
from agents import function_tool

# Simple tools for the order agent
@function_tool
def lookup_order(order_id: str) -> str:
    """Look up order details by ID."""
    orders = {
        "ORD-001": "Laptop, Status: Shipped, ETA: Tomorrow",
        "ORD-002": "Mouse, Status: Delivered"
    }
    return orders.get(order_id, f"Order {order_id} not found")

# Order specialist with tools
order_agent = Agent(
    name="OrderSpecialist",
    instructions="You help customers with order inquiries. Use the lookup_order tool to find order details.",
    model="gpt-4.1",
    tools=[lookup_order],
    handoff_description="Hand off for order status, shipping, and delivery questions"
)

# General info agent (no special tools needed)
general_agent = Agent(
    name="GeneralInfo",
    instructions="You answer general questions about the company, policies, and products.",
    model="gpt-4.1",
    handoff_description="Hand off for general questions about policies, products, or the company"
)

# Triage routes to specialists
customer_service = Agent(
    name="CustomerService",
    instructions="Route customers to the right specialist based on their question.",
    model="gpt-4.1",
    handoffs=[order_agent, general_agent]
)

In [12]:
# Order question → handoff to order specialist → uses lookup_order tool
result = Runner.run_sync(customer_service, "Where is my order ORD-001?")
print(result.final_output)

Your order (ORD-001) for a laptop has been shipped and is expected to arrive tomorrow. If you need tracking details or more information, please let me know!


In [13]:
# General question → handoff to general info agent
result = Runner.run_sync(customer_service, "What's your return policy?")
print(result.final_output)

Our return policy typically allows you to return most items within a certain period after purchase, as long as they are in their original condition and packaging. To give you precise details (such as the return window, eligibility, and process), could you please specify what item or product you’re referring to? Alternatively, let me know if you're asking about online or in-store purchases.

If you need instructions or have a specific situation (like a defective item), let me know so I can provide more tailored information!


## Key Takeaways

1. **Handoffs** transfer control completely - specialist takes over
2. **Agents-as-Tools** keep manager in control - manager synthesizes results
3. **`handoff_description`** tells the routing agent when to use each handoff
4. **`agent.as_tool()`** turns any agent into a callable tool
5. **Combine patterns** for complex systems (triage + tools)

### Decision Cheat Sheet

| Question | If Yes → | If No → |
|----------|----------|----------|
| Does the specialist complete the whole task? | Handoff | Agents-as-Tools |
| Need to combine results from multiple agents? | Agents-as-Tools | Handoff |
| Should user interact with specialist directly? | Handoff | Agents-as-Tools |

Next up: **Guardrails, Sessions, and Tracing** - making your agents production-ready.